In [20]:
import pandas as pd # 引用套件並縮寫為 pd  
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import math

from sklearn.model_selection import train_test_split       #split data
from sklearn.experimental import enable_iterative_imputer  #deal with missing data
from sklearn.impute import IterativeImputer                #deal with missing data
from sklearn.model_selection import KFold                  #deal with validation
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

import seaborn as sns                                      #deal with heatmap

In [3]:
#read data
df99 = pd.read_csv('data/akari_data.csv', low_memory=False)

In [4]:
df = df99.replace(99,np.NaN) #replace Nan to missing data
df = df[df['fl1']==1] #selecct the galaxy

In [5]:
train_flux=['GCm','RCm','ICm','ZCm','YCm','N2m','N3m','N4m','S7m','S9m','S11m','L15m','L18m','L24m','w1m','w2m','ir1','ir2','specz','photz']
target_flux=['m250','m350','m500']
train_error=['GCme','RCme','ICme','ZCme','YCme','N2me','N3me','N4me','S7me','S9me','S11me','L15me','L18me','L24me','w1me','w2me','ir1e','ir2e']
target_error=['m250e','m350e','m500e']

In [6]:
df_train=df[train_flux]
df_target=df[target_flux]
df_trainerr=df[train_error]
df_targeterr=df[target_error]
# df_train.to_csv('data/training set.csv')
# df_target.to_csv('data/target set.csv')
# df_trainerr.to_csv('data/train error.csv')
# df_targeterr.to_csv('data/target error.csv')

In [7]:
df_target250=df_target['m250']
df_target350=df_target['m350']
df_target500=df_target['m500']

df_target250=df_target250.dropna()
df_target350=df_target350.dropna()
df_target500=df_target500.dropna()

df_target250=df_target250[df_target250<14]
df_target350=df_target350[df_target350<14.2]
df_target500=df_target500[df_target500<13.8]

# df_target250.to_csv('data/target250.csv')
# df_target350.to_csv('data/target350.csv')
# df_target500.to_csv('data/target500.csv')

df_targeterr250=df_targeterr['m250e'].loc[df_target250.index]
df_targeterr350=df_targeterr['m350e'].loc[df_target350.index]
df_targeterr500=df_targeterr['m500e'].loc[df_target500.index]

# df_targeterr250.to_csv('data/targeterr250.csv')
# df_targeterr350.to_csv('data/targeterr350.csv')
# df_targeterr500.to_csv('data/targeterr500.csv')

In [38]:
df_target_mix=pd.concat([df_target250,df_target350,df_target500],axis=1)
# df_target_mix
df_train=df_train[train_flux[:-2]]
df_train_imp=df_train.loc[df_target_mix.index]

In [40]:
#missing data
imp = IterativeImputer(max_iter=1000, random_state=0,sample_posterior=True)
imp.fit(df_train_imp)

IterativeImputer(max_iter=1000, random_state=0, sample_posterior=True)

In [43]:
df_train_imp=imp.transform(df_train_imp) #type is np.array
df_train_imp=pd.DataFrame(data=df_train_imp,columns=train_flux[:-2],index=pd.Index(df_target_mix.index))

In [55]:
df_test_imp=imp.transform(df_train)
df_test_imp=pd.DataFrame(data=df_test_imp,columns=train_flux[:-2])

In [47]:
# df_train_imp.to_csv('data/train_imp.csv')
# df_test_imp.to_csv('data/test_imp.csv')

In [46]:
df_train250=df_train_imp.reindex(pd.Index(df_target250.index))
df_train350=df_train_imp.reindex(pd.Index(df_target350.index))
df_train500=df_train_imp.reindex(pd.Index(df_target500.index))
# df_train250.to_csv('data/train250.csv')
# df_train350.to_csv('data/train350.csv')
# df_train500.to_csv('data/train500.csv')

In [48]:
df_trainerr_imp=df_trainerr.loc[df_target_mix.index]
#missing data
imperr = IterativeImputer(max_iter=1000, random_state=0,sample_posterior=True)
imperr.fit(df_trainerr_imp)

IterativeImputer(max_iter=1000, random_state=0, sample_posterior=True)

In [49]:
df_trainerr_imp=imperr.transform(df_trainerr_imp)
df_trainerr_imp=pd.DataFrame(data=df_trainerr_imp,columns=train_error,index=pd.Index(df_target_mix.index))

In [53]:
df_testerr_imp=imperr.transform(df_trainerr)
df_testerr_imp=pd.DataFrame(data=df_testerr_imp,columns=train_error)

In [16]:
df_testerr_imp

,GCme,RCme,ICme,ZCme,YCme,N2me,N3me,N4me,S7me,S9me,S11me,L15me,L18me,L24me,w1me,w2me,ir1e,ir2e
0,0.014,0.005,0.010000,0.017000,0.021000,0.065000,0.059000,0.079000,0.854306,0.089000,0.093000,4.238298,4.481342,7.226003,0.058000,0.122000,0.054000,0.048000
1,0.152,0.174,0.279000,0.159657,0.247000,-1.402775,0.115000,0.121000,5.091689,6.999206,8.838239,10.043947,10.065021,11.305695,0.206000,0.218000,0.711702,0.767933
2,0.004,0.001,0.002000,0.004000,0.005000,0.022000,0.022000,0.032000,0.075418,0.120000,1.758754,2.684857,0.126000,4.850728,0.032000,0.053000,0.025000,0.024000
3,0.007,0.004,0.004000,0.007000,0.006000,0.082000,0.081000,0.095000,0.828889,1.740827,2.247648,0.109000,2.993678,5.942613,0.064000,0.182000,0.054000,0.069000
4,0.079,0.040,0.047000,0.051000,0.058000,0.072000,0.052000,0.067000,5.511729,7.310214,8.697908,9.247705,9.087889,11.638285,0.044000,0.101000,0.043000,0.051000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67705,0.061,0.305,0.157000,0.277000,1.536292,6.168195,7.497905,8.224788,5.292087,3.810953,4.608219,4.087843,0.027000,0.092000,7.242014,6.956045,7.361164,7.590189
67706,0.038,0.141,0.153000,0.183000,1.177489,5.706378,7.641428,8.091109,6.150587,4.652311,5.151360,4.730618,0.058000,5.299215,6.679502,6.334393,7.448606,7.325924
67707,0.108,0.219,2.164593,3.705124,4.780667,11.771469,14.500312,16.385886,17.079678,15.766571,15.382496,16.244719,16.990519,16.963516,14.600050,16.053041,14.977963,15.882312
67708,0.143,0.090,-0.147925,0.188000,1.070564,5.949226,7.769256,8.354935,6.159147,4.059950,4.842302,3.809150,0.083000,5.414704,6.528220,5.491381,7.000415,6.935435


In [54]:
# df_trainerr_imp.to_csv('data/trainerr_imp.csv')
# df_testerr_imp.to_csv('data/testerr_imp.csv')

In [52]:
df_trainerr250=df_trainerr_imp.loc[df_target250.index]
df_trainerr350=df_trainerr_imp.loc[df_target350.index]
df_trainerr500=df_trainerr_imp.loc[df_target500.index]
# df_trainerr250.to_csv('data/trainerr250.csv')
# df_trainerr350.to_csv('data/trainerr350.csv')
# df_trainerr500.to_csv('data/trainerr500.csv')

In [248]:
z=df[train_flux[-2:]]
specz=z[z['specz']!=-1]['specz']
photz=z[z['specz']==-1]['photz']
z=pd.concat([specz,photz])
z=z.sort_index()
z=pd.DataFrame(z,columns=['z'])
# z.to_csv('data/z.csv')

In [249]:
df_train_imp_z=pd.concat([df_train_imp, z], axis=1)

In [254]:
df_train250z=df_train_imp_z.reindex(pd.Index(df_target250.index))
df_train350z=df_train_imp_z.reindex(pd.Index(df_target350.index))
df_train500z=df_train_imp_z.reindex(pd.Index(df_target500.index))

# df_train250z.to_csv('data/train250 with z.csv')
# df_train350z.to_csv('data/train350 with z.csv')
# df_train500z.to_csv('data/train500 with z.csv')